In [2]:
from dotenv import load_dotenv

In [5]:
import json
import os
from typing import Literal, Union
from pydantic import BaseModel, Field

import fixpoint
from fixpoint.agents.openai import OpenAIClients

In [6]:
class SportsCar(BaseModel):
    explanation: str = Field(description="An explanation of why you picked the given car")
    car: Union[Literal['Porsche 911'], Literal['Honda Fit'], Literal['Ford F-150']] = Field(
        description="the make and model of the car"
    )

In [7]:
agent = fixpoint.agents.oai.OpenAI(
    model_name='gpt-3.5-turbo',
    openai_clients=OpenAIClients.from_api_key(os.environ['OPENAI_API_KEY']),
)

In [8]:
compl = agent.chat.completions.create(
    messages=[
        {"role": "user",
         "content": "Can you help me pick out a new sports car? I often need to haul a lot of cargo"},
    ],
    response_model=SportsCar
)

In [9]:
tool_call_dict = json.loads(compl.choices[0].message.tool_calls[0].function.arguments)
print(tool_call_dict)

{'explanation': 'It is important to have a sports car that can haul cargo efficiently', 'car': 'Ford F-150'}


In [10]:
compl.fixp.structured_output

SportsCar(explanation='It is important to have a sports car that can haul cargo efficiently', car='Ford F-150')

In [23]:
import json
import os
from typing import Literal, Union

from dotenv import load_dotenv

import fixpoint
from fixpoint.agents.callbacks import TikTokenLogger
from fixpoint.agents.openai import OpenAIClients
from fixpoint.analyze.memory import DataframeMemory
from fixpoint.utils.messages import amsg, smsg, umsg

In [10]:
from pydantic import BaseModel, Field

In [5]:
import logging
import sys
from fixpoint.logging import logger
logging.basicConfig(stream=sys.stdout)
logger.setLevel(logging.INFO)
logger.info("Info mode is on?")

INFO:fixpoint:Info mode is on?


In [3]:
assert load_dotenv('../.env')

In [8]:
model_name = 'gpt-3.5-turbo'
agent_mem = DataframeMemory()
tokenlogger = TikTokenLogger(model_name)
agent = fixpoint.agents.oai.OpenAI(
    model_name=model_name,
    openai_clients=OpenAIClients.from_api_key(os.environ['OPENAI_API_KEY']),
    memory=agent_mem,
    pre_completion_fns=[tokenlogger.tiktoken_logger]
)

In [11]:
class SportsCar(BaseModel):
    explanation: str = Field(description="An explanation of why you picked the given car")
    car: Union[Literal['Porsche 911'], Literal['Honda Fit'], Literal['Ford F-150']] = Field(
        description="the make and model of the car"
    )

In [16]:
compl = agent.chat.completions.create(
    messages=[
        umsg("Can you help me pick out a new sports car? I often need to haul a lot of cargo"),
    ],
    response_model=SportsCar
)

INFO:fixpoint:Total input tokens: 43


In [17]:
len(agent_mem.memories())

2

In [26]:
tool_call_dict = json.loads(compl.choices[0].message.tool_calls[0].function.arguments)
print(tool_call_dict)

{'explanation': "Considering your need to haul a lot of cargo, I recommend a Ford F-150. It's a versatile and powerful truck that can handle both performance and practicality.", 'car': 'Ford F-150'}


In [27]:
compl.fixp.structured_output

SportsCar(explanation="Considering your need to haul a lot of cargo, I recommend a Ford F-150. It's a versatile and powerful truck that can handle both performance and practicality.", car='Ford F-150')

In [11]:
print(agent_mem.to_str())

system: You are a friendly AI customer support agent. You chat with a user to answer their questions
user: Can you help me pick out a new sports car?
assistant: None


In [12]:
compl = agent.chat.completions.create(
    messages=[
        smsg("You are a friendly AI customer support agent. You chat with a user to answer their questions"),
        umsg("Can you help me pick out a new sports car? I have a family of 6, should I get a Porsche or a Dodge Caravan?"),
    ],
    response_model=SportsCar
)

INFO:fixpoint:Total input tokens: 53


In [13]:
compl.fixp.structured_output

SportsCar(explanation='If you have a family of 6, a Dodge Caravan would be more suitable as it provides more seating capacity and space for your family members.', car='Dodge Caravan')

In [14]:
compl.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z61wwiaS6GPTwwA5LaBJdmu6', function=Function(arguments='{"explanation":"If you have a family of 6, a Dodge Caravan would be more suitable as it provides more seating capacity and space for your family members.","car":"Dodge Caravan"}', name='SportsCar'), type='function')]))

In [15]:
df = agent_mem.to_dataframe()

In [16]:
df

,turn_id,role,content,structured_output,workflow_id
0,0,system,You are a friendly AI customer support agent. ...,None,None
1,0,user,Can you help me pick out a new sports car?,None,None
2,0,assistant,None,"{'explanation': 'Sure, I'd be happy to help yo...",None
3,1,system,You are a friendly AI customer support agent. ...,None,None
4,1,user,Can you help me pick out a new sports car? I h...,None,None
5,1,assistant,None,"{'explanation': 'If you have a family of 6, a ...",None


In [17]:
df[df['turn_id'] == 1]

,turn_id,role,content,structured_output,workflow_id
3,1,system,You are a friendly AI customer support agent. ...,None,None
4,1,user,Can you help me pick out a new sports car? I h...,None,None
5,1,assistant,None,"{'explanation': 'If you have a family of 6, a ...",None


In [19]:
cmpl = agent_mem.memory()[-1].completion

In [20]:
str(cmpl.fixp.structured_output)

"explanation='If you have a family of 6, a Dodge Caravan would be more suitable as it provides more seating capacity and space for your family members.' car='Dodge Caravan'"

In [21]:
cmpl.fixp.structured_output.dict()

{'explanation': 'If you have a family of 6, a Dodge Caravan would be more suitable as it provides more seating capacity and space for your family members.',
 'car': 'Dodge Caravan'}

In [22]:
agent_mem.to_dataframe()

,turn_id,role,content,structured_output,workflow_id
0,0,system,You are a friendly AI customer support agent. ...,None,None
1,0,user,Can you help me pick out a new sports car?,None,None
2,0,assistant,None,"{'explanation': 'Sure, I'd be happy to help yo...",None
3,1,system,You are a friendly AI customer support agent. ...,None,None
4,1,user,Can you help me pick out a new sports car? I h...,None,None
5,1,assistant,None,"{'explanation': 'If you have a family of 6, a ...",None
